In [7]:
import os
import glob
from bs4 import BeautifulSoup, NavigableString

In [5]:
root_dir = '3.6/manual'
html_dir = root_dir + '/raw_html'
html_files = []

html_files = glob.glob(f'{html_dir}/**/*.html', recursive=True)

print(len(html_files))
html_files[45]

1998


'3.6/manual/raw_html/interface/controls/nodes/introduction.html'

In [10]:
def read_page(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    return html_content

def path_to_title(path):
    parts = path.split('/')[3:]
    formatted_parts = []
    for part in parts:
        # Remove file extension, replace underscores with spaces, capitalize
        part = part.split('.')[0].replace('_', ' ').title()
        formatted_parts.append(part)
    
    return ' > '.join(formatted_parts)

def path_to_slug(path):
    parts = path.split('/')[3:]
    formatted_parts = []
    for part in parts:
        # Remove file extension, replace underscores with spaces, capitalize
        part = part.split('.')[0]
        formatted_parts.append(part)
    
    return '-'.join(formatted_parts)


In [48]:
def clean_doc_page(path):
    html = read_page(path)
    soup = BeautifulSoup(html, 'html.parser')
    
    title = path_to_title(path)
    main_document = soup.find('div', {'role': 'main', 'class': 'document'})
    breadcrumbs = soup.find('ul', class_='wy-breadcrumbs')
    
    #TODO: Why yes i know i loop like 4 times through the page for no reason. 
    def clean_tags(tag):
        tag.attrs = {}

        # Delete all <a class="headerlink"> tags
        for headerlink in tag.find_all('a', class_='headerlink'):
            headerlink.decompose()
            
        # Replace each 'span' with just its text content
        for span in tag.find_all('span'):
            span.replace_with(NavigableString(span.text))
        
        for sub_tag in tag.find_all(True):
           # no classes on tags
            if 'class' in sub_tag.attrs:
                del sub_tag['class']
            if 'style' in sub_tag.attrs:
                del sub_tag['style']

        for sub_tag in tag.find_all(True):
            # no tags without text
            if not sub_tag.get_text(strip=True):
                sub_tag.decompose()

    
    # Apply the cleaning to found elements
    if main_document:
        clean_tags(main_document)
    if breadcrumbs:
        clean_tags(breadcrumbs)
    
    # Return the cleaned elements
    return title, main_document, breadcrumbs


In [62]:
def cleaned_to_page(title, main_document, breadcrumbs):
    main_document_str = str(main_document)
    breadcrumbs_str = str(breadcrumbs)

    html_template = f"""
    <!DOCTYPE html>
    <html xmlns="http://www.w3.org/1999/xhtml">
    <head><title>{title}</title></head>
    <body>{breadcrumbs_str}{main_document_str}</body>
    </html>
    """
    soup = BeautifulSoup(html_template, 'xml')

    #print(soup.prettify())
    return str(soup)


In [63]:
file = html_files[46]
print(file)

def process_page(path):
    out = root_dir + '/cleaned_html/' + path_to_slug(path) + '.html'
    title, main, bread = clean_doc_page(path)
    result = cleaned_to_page(title,main,bread)
    with open(out, 'w', encoding='utf-8') as file:
        file.write(result)
        
for path in html_files:
    process_page(path)

3.6/manual/raw_html/interface/controls/templates/list_view.html


FileNotFoundError: [Errno 2] No such file or directory: '3.6/manual/raw_html/contribute/translations/style_guide.html'